# Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [13]:
# imports
import numpy as np
import cv2
import glob
import pickle
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from calibration import Calibration
from utils import display
import preprocessing

### Lane detection 

In [ ]:
search_params = {
    'nwindows': 9, # Choose the number of sliding windows
    'margin': 100, # Set the width of the windows +/- margin
    'minpix': 50   # Set minimum number of pixels found to recenter window
}

def find_lane_pixels(img, params):    
    # Create histogram of image binary activations
    histogram = np.sum(img[img.shape[0]//2:,:], axis=0)
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    lane_gap = rightx_base-leftx_base
    
    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(img.shape[0]//params['nwindows'])
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    # Step through the windows one by one
    for window in range(params['nwindows']):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        # Find the four below boundaries of the window
        win_xleft_low = leftx_current-params['margin']
        win_xleft_high = leftx_current+params['margin']
        win_xright_low = rightx_current-params['margin']
        win_xright_high = rightx_current+params['margin']
        
        #Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                          (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                          (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        # If you found > minpix pixels, recenter next window
        # (`right` or `leftx_current`) on their mean position
        if len(good_left_inds) > params['minpix']:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > params['minpix']:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
            
    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return leftx, lefty, rightx, righty, lane_gap

_,_,_,_,lane_gap = find_lane_pixels(output_mask, search_params)
print(lane_gap)

In [ ]:
def fit_polynomial(img, params):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, lane_gap = find_lane_pixels(img, params)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    return left_fit, right_fit

def plot_ploy_lines(img, left_fit, right_fit):
    # Generate x and y values for plotting
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    plt.imshow(img, cmap='gray')
    plt.plot(left_fitx, ploty, color='red')
    plt.plot(right_fitx, ploty, color='red')
    
left_fit, right_fit = fit_polynomial(output_mask, search_params)
plot_ploy_lines(output_mask, left_fit, right_fit)

### Get curvature

In [ ]:
def measure_curvature(y, left_fit, right_fit):
    '''
    Calculates the curvature of polynomial functions.
    '''
        
    # calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*y*left_fit[0]+left_fit[1])**2)**(3/2))/np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*y*right_fit[0]+right_fit[1])**2)**(3/2))/np.absolute(2*right_fit[0])
    
    return left_curverad, right_curverad

measure_curvature(719, left_fit, right_fit)

In [ ]:
# meters per pixels
xm = 3.7/lane_gap
ym = 30/720

def cvt_line_meters(line):
    new_line = np.copy(line)
    new_line[0] *= xm/(ym**2)
    new_line[1] *= xm/ym
    return new_line

left_fit_m = cvt_line_meters(left_fit)
right_fit_m = cvt_line_meters(right_fit)

measure_curvature(719*ym, left_fit_m, right_fit_m)